In [1]:
%%capture

!pip install sentence-transformers numpy py_vncorenlp rouge_score bert_score vncorenlp evaluate
!pip install -U sentence-transformers

In [2]:
from sentence_transformers import SentenceTransformer
import numpy as np
import pandas as pd
import os

2025-11-16 18:51:50.175330: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763319110.359633      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763319110.409491      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

In [3]:
#Thiết lập cho pandas
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)
pd.set_option("display.max_colwidth", None)

In [4]:
%%capture

!sudo apt-get update
!sudo apt-get install openjdk-8-jdk -y
!java -version

In [5]:
!mkdir vncorenlp

In [6]:
%%capture

import py_vncorenlp

py_vncorenlp.download_model(save_dir='/kaggle/working/vncorenlp',)
nlp = py_vncorenlp.VnCoreNLP(save_dir='/kaggle/working/vncorenlp')

--2025-11-16 18:52:33--  https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/VnCoreNLP-1.2.jar
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 27412703 (26M) [application/octet-stream]
Saving to: ‘VnCoreNLP-1.2.jar’

     0K .......... .......... .......... .......... ..........  0% 1.22M 21s
    50K .......... .......... .......... .......... ..........  0% 3.49M 14s
   100K .......... .......... .......... .......... ..........  0% 1.81M 14s
   150K .......... .......... .......... .......... ..........  0% 6.49M 12s
   200K .......... .......... .......... .......... ..........  0% 7.21M 10s
   250K .......... .......... .......... .......... ..........  1% 2.41M 10s
   300K .......... .......... .......... .......... ..........  1% 10.8M 9s
   3

2025-11-16 18:52:47 INFO  WordSegmenter:24 - Loading Word Segmentation model
2025-11-16 18:52:47 INFO  PosTagger:23 - Loading POS Tagging model
2025-11-16 18:52:51 INFO  NerRecognizer:34 - Loading NER model
2025-11-16 18:53:04 INFO  DependencyParser:32 - Loading Dependency Parsing model


In [7]:
import py_vncorenlp
from vncorenlp import VnCoreNLP
from collections import Counter
import math
from nltk.translate.bleu_score import SmoothingFunction
from rouge_score import rouge_scorer
from bert_score import score as bert_score
import torch
import random

In [8]:
def set_SEED():
    SEED = 42
    random.seed(SEED)
    np.random.seed(SEED)
    torch.manual_seed(SEED)
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)
    torch.backends.cudnn.enabled = False
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True
    os.environ["PYTHONHASHSEED"] = str(SEED)

set_SEED()

In [9]:
filename = "/kaggle/input/vietnamhistory-dataset/vietnamese-stopwords.txt"  # Điều chỉnh đường dẫn nếu cần

# Đọc stopwords từ file txt
with open(filename, 'r', encoding='utf-8') as f:
    list_stopwords = f.read().splitlines()

print(f"Số lượng stopwords đã tải: {len(list_stopwords)}")

Số lượng stopwords đã tải: 3884


In [10]:

class StopwordRemover:
    def __init__(self, stopwords):
        self.stopwords = set(stopwords)

    def remove_stopwords(self, text):
        pre_text = []
        words = text.split()
        for word in words:
            if word.lower() not in self.stopwords:
                pre_text.append(word)
        return ' '.join(pre_text)

class VNPreprocessor:
    def __init__(self, nlp):
        self.nlp = nlp

    def preprocess(self, text):
        segmented_sentences = self.nlp.word_segment(text)
        processed_text = ' '.join(segmented_sentences)
        return processed_text

preprocessor = VNPreprocessor(nlp)
stopword_remover = StopwordRemover(list_stopwords)

In [11]:
csv_filename = '/kaggle/input/vietnamhistory-dataset/train_data.csv'  # Thay đổi đường dẫn này

if not os.path.exists(csv_filename):
    raise FileNotFoundError(f"CSV file not found at {csv_filename}. Vui lòng kiểm tra lại đường dẫn.")

# Đọc CSV
df = pd.read_csv(csv_filename)
df = df[['period', 'context']]
df = df.drop_duplicates()
print(f"Số lượng bản ghi trong dữ liệu: {len(df)}")

# df = df.head()


print("Chuyển context sang chữ thường ...")
df['context'] = df['context'].str.lower()

print("Tiền xử lý các context bằng VNCoreNLP...")
df['processed_context'] = df['context'].apply(preprocessor.preprocess)

print("Loại bỏ stopwords từ các context cho BM25...")
df['processed_context_bm25'] = df['processed_context'].apply(stopword_remover.remove_stopwords)

Số lượng bản ghi trong dữ liệu: 3515
Chuyển context sang chữ thường ...
Tiền xử lý các context bằng VNCoreNLP...
Loại bỏ stopwords từ các context cho BM25...


In [12]:
df.head()

,period,context,processed_context,processed_context_bm25
0,Giai đoạn 1951 - 1954,"từ những ngày đầu của cuộc kháng chiến, đảng và chính phủ ban hành nhiều chính sách về ruộng đất nhằm mang lại những quyền lợi bước đầu cho nông dân. tuy vậy, chính sách ruộng đất của đảng và chính phủ thực hiện trong những năm đầu kháng chiến chưa phải là cải cách ruộng đất triệt để, mà trước hết chỉ ""nhằm hạn chế sự bóc lột phong kiến"", tập trung giải quyết những vấn đề giảm tô, giảm tức, chia ruộng đất của thực dân pháp và việt gian cho nông dân nghèo, sử dụng hợp lý ruộng đất vắng chủ, ruộng đất bỏ hoang, chia công điền công thổ, phát động phong trào hiến điền, thực hiện chính sách thuế công bằng dân chủ.","từ những ngày đầu của cuộc kháng_chiến , đảng và chính_phủ ban_hành nhiều chính_sách về ruộng_đất nhằm mang lại những quyền_lợi bước_đầu cho nông_dân . tuy_vậy , chính_sách ruộng_đất của đảng và chính_phủ thực_hiện trong những năm đầu kháng_chiến chưa phải là cải_cách_ruộng_đất triệt_để , mà trước_hết chỉ "" nhằm hạn_chế sự bóc_lột phong_kiến "" , tập_trung giải_quyết những vấn_đề giảm tô , giảm tức , chia ruộng_đất của thực_dân pháp và việt_gian cho nông_dân nghèo , sử_dụng hợp_lý ruộng_đất vắng chủ , ruộng_đất bỏ_hoang , chia công_điền công_thổ , phát_động phong_trào hiến điền , thực_hiện chính_sách thuế công_bằng dân_chủ .","đầu kháng_chiến , đảng chính_phủ ban_hành chính_sách ruộng_đất quyền_lợi bước_đầu nông_dân . , chính_sách ruộng_đất đảng chính_phủ đầu kháng_chiến cải_cách_ruộng_đất triệt_để , "" hạn_chế bóc_lột phong_kiến "" , giải_quyết tô , tức , chia ruộng_đất thực_dân pháp việt_gian nông_dân nghèo , hợp_lý ruộng_đất vắng chủ , ruộng_đất bỏ_hoang , chia công_điền công_thổ , phát_động phong_trào hiến điền , chính_sách thuế công_bằng dân_chủ ."
1,Giai đoạn 1951 - 1954,kết quả là số thu năm 1952 đã đảm bảo được 78% số chi và năm 1953 lần đầu tiên kể từ cách mạng tháng tám số thu chẳng những hoàn toàn đảm bảo số chi mà còn vượt số chi tới 16%.,kết_quả là số thu năm 1952 đã đảm_bảo được 78% số chi và năm 1953 lần đầu_tiên kể từ cách_mạng_tháng_tám số thu chẳng_những hoàn_toàn đảm_bảo số chi mà_còn vượt số chi tới 16% .,kết_quả thu 1952 78% chi 1953 cách_mạng_tháng_tám thu chi mà_còn chi 16% .
2,Giai đoạn 1951 - 1954,"tại nam bộ: trước nguy cơ suy sụp ngày càng gia tăng của thực dân pháp, đế quốc mỹ nhanh chóng tranh thủ thời cơ bắt đầu thực hiện âm mưu trực tiếp can thiệp sâu vào đông dương. các chuyến tàu viện trợ quân sự của mỹ cập bến sài gòn ngày càng nhiều. đối với miền đông nam bộ, nỗ lực hoạt động của địch nằm trong phạm vi thực hiện chủ trương, phối hợp tiến công bằng cả kinh tế, chính trị, quân sự nhằm đánh mạnh vào lực lượng du kích, chiến lược chiến tranh du kích của ta, cướp vét và tận dụng dự trữ, thực hiện chính sách ""lấy chiến tranh nuôi chiến tranh"" và bao vây cô lập khu 7 bằng cách ngăn chặn, kiểm soát chặt chẽ khu 8... từ đầu năm 1951 trở đi, địch ráo riết thiết lập các điểm chốt chặn trên dọc hành lang đức hòa - hóc môn, kiểm soát chặt đường biển, đường sông vàm cỏ, đường số 10 và số 1, cắt đứt đường giao thông tiếp viện của ta từ miền tây lên miền đông; tăng cường lực lượng bảo vệ các đường giao thông chiến lược nối thông với tây nguyên (đường 14, 20), với cực nam trung bộ (đường 1 và đường xe lửa).","tại nam_bộ : trước nguy_cơ suy_sụp ngày_càng gia_tăng của thực_dân pháp , đế_quốc mỹ nhanh_chóng tranh_thủ thời_cơ bắt_đầu thực_hiện âm_mưu trực_tiếp can_thiệp sâu vào đông dương . các chuyến tàu viện_trợ quân_sự của mỹ cập bến sài_gòn ngày_càng nhiều . đối_với miền đông nam_bộ , nỗ_lực hoạt_động của địch nằm trong phạm_vi thực_hiện chủ_trương , phối_hợp tiến_công bằng cả kinh_tế , chính_trị , quân_sự nhằm đánh mạnh vào lực_lượng du_kích , chiến_lược chiến_tranh_du_kích của ta , cướp vét và tận_dụng dự_trữ , thực_hiện chính_sách "" lấy chiến_tranh nuôi chiến_tranh "" và bao_vây cô_lập khu 7 bằng cách ngăn_chặn , kiểm_soát chặt_chẽ khu 8 ... từ đầu năm 1951 trở đi , địch ráo_riết thiết_lập các điểm chốt chặn trên

In [13]:
%%capture

!pip install weaviate-client

In [14]:
import weaviate
from weaviate.classes.init import Auth

# ---- Thay bằng cluster của bạn ----
WEAVIATE_URL = "v1h18i2sce1cl42acvvpa.c0.asia-southeast1.gcp.weaviate.cloud"
WEAVIATE_API_KEY = "bTNhaG8xcHVlcjVtQXNMbV9UYmRKZEtFMzdTanhFWjlhUFBzWnVzTDl6WHFOUmY1REtkRHZQby9BNDB3PV92MjAw"


client = weaviate.connect_to_weaviate_cloud(
    cluster_url=WEAVIATE_URL,
    auth_credentials=Auth.api_key(WEAVIATE_API_KEY),
)

print("Connected:", client.is_ready())

Connected: True


In [15]:
from weaviate.classes.config import Property, DataType

client.collections.delete_all()

client.collections.create(
    name="History",
    vectorizer_config=None,
    properties=[
        Property(name="context", data_type=DataType.TEXT),
        Property(name="period",  data_type=DataType.TEXT)
    ]
)
print("Collection 'History' created!")

Collection 'History' created!


In [16]:
print(len(df))

3515


In [17]:
from weaviate.classes.data import DataObject
from weaviate.classes.config import Property, DataType
from tqdm.auto import tqdm # Import tqdm để theo dõi tiến trình
import math
import gc

# --- Load embedding model ---
embed_model = SentenceTransformer("/kaggle/input/bge-m3-lora-embedding-models/transformers/default/1")

print("\n--- Bắt đầu chèn dữ liệu từ DataFrame 'df' vào Weaviate ---")

# --- Thiết lập kích thước lô an toàn ---
BATCH_SIZE = 500 # Bạn có thể điều chỉnh con số này dựa trên dung lượng bộ nhớ của bạn

print("\n--- Bắt đầu chèn dữ liệu theo lô nhỏ vào Weaviate ---")

history_collection = client.collections.get("History")

num_batches = math.ceil(len(df) / BATCH_SIZE)
print(f"Tổng số bản ghi: {len(df)}. Tổng số lô (Batch): {num_batches}.")

# Sử dụng tqdm để theo dõi tiến trình
for i in tqdm(range(0, len(df), BATCH_SIZE), desc="Processing Batches"):
    
    # 1. Lấy lô dữ liệu
    batch_df = df.iloc[i : i + BATCH_SIZE]
    
    # 2. Mã hóa vector cho lô hiện tại
    # Lưu ý: Sử dụng context đã được tiền xử lý (processed_context) cho embedding
    batch_contexts = batch_df['processed_context'].tolist()
    batch_vectors = embed_model.encode(batch_contexts, convert_to_tensor=True).cpu().tolist()
    
    data_objects_to_insert = []
    
    # 3. Chuẩn bị DataObject
    # for index, row in batch_df.iterrows():
    for j, (index, row) in enumerate(batch_df.iterrows()):
        
        # Lấy vector tương ứng từ danh sách vectors của lô này
        # vector_index_in_batch = index - i 
        
        properties = {
            "context": row['processed_context'],
            "period": row['period']
        }
        
        data_objects_to_insert.append(
            DataObject(
                properties=properties,
                # vector=batch_vectors[vector_index_in_batch]
                vector=batch_vectors[j]
                
            )
        )

    # 4. Chèn lô vào Weaviate
    response = history_collection.data.insert_many(data_objects_to_insert)

    # 5. Kiểm tra lỗi của lô hiện tại
    if response.errors:
        print(f"\n⚠️ Lỗi xảy ra ở Lô {i // BATCH_SIZE + 1} (từ index {i} đến {i + BATCH_SIZE}): {len(response.errors)} lỗi.")
        # Nếu muốn dừng lại khi có lỗi: raise Exception("Lỗi chèn batch.")

    # 5. Giải phóng bộ nhớ
    del batch_vectors
    torch.cuda.empty_cache() # Nếu dùng GPU
    gc.collect()
        
print("--- Kết thúc chèn dữ liệu ---")

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 


--- Bắt đầu chèn dữ liệu từ DataFrame 'df' vào Weaviate ---

--- Bắt đầu chèn dữ liệu theo lô nhỏ vào Weaviate ---
Tổng số bản ghi: 3515. Tổng số lô (Batch): 8.


Processing Batches:   0%|          | 0/8 [00:00<?, ?it/s]

--- Kết thúc chèn dữ liệu ---


In [18]:

# --- Thay thế cho dòng client.close() đã chạy trước đó ---
client.connect() 
print("Client reconnected:", client.is_ready())

# --- Chuẩn bị truy vấn ---
query_text = "Trước tình hình âm mưu của đế quốc mỹ thì nam bộ làm gì?"

# 1. Mã hóa câu truy vấn thành vector
query_vector = embed_model.encode([query_text])[0].tolist()

# 2. Lấy collection "History"
history_collection = client.collections.get("History")

# 3. Thực hiện truy vấn near_vector
# Quan trọng: Cần yêu cầu trả về siêu dữ liệu (metadata)
response = history_collection.query.near_vector(
    near_vector=query_vector,
    return_properties=["context", "period"], 
    # Yêu cầu trả về distance
    return_metadata=weaviate.classes.query.MetadataQuery(distance=True), 
    limit=1 
)

## 4. In kết quả
print(f"\n--- Kết quả tìm kiếm cho truy vấn: '{query_text}' ---")
if response.objects:
    for obj in response.objects:
        print(f"**Period:** {obj.properties['period']}")
        print(f"**Context:** {obj.properties['context']}")
        # Sửa lỗi: Truy cập distance thông qua .metadata
        print(f"**Distance (độ tương tự):** {obj.metadata.distance:.4f}") 
        print("-" * 20)
else:
    print("Không tìm thấy kết quả liên quan.")

# Đừng quên đóng kết nối nếu bạn chưa làm
client.close()

Client reconnected: True

--- Kết quả tìm kiếm cho truy vấn: 'Trước tình hình âm mưu của đế quốc mỹ thì nam bộ làm gì?' ---
**Period:** Giai đoạn 1965 -1975
**Context:** chính từ cách nhìn đó , bộ_chính_trị đã nhận_định : chúng_ta đang đứng trước thời_cơ lớn để thừa_thắng tiến lên , đẩy_mạnh tiến_công toàn_diện và liên_tục , tiến lên một bước mới rất cơ_bản , đánh_bại ý_chí xâm_lược của mỹ . nhiệm_vụ trước_mắt được bộ_chính_trị đề ra là : động_viên sự nỗ_lực cao nhất cho toàn đảng , toàn quân , toàn dân trên cả hai miền , phát_huy thắng_lợi đã đạt được , tiếp_tục đẩy_mạnh tổng_công_kích , tổng_khởi_nghĩa , đẩy_mạnh ba_mũi_giáp_công , kết_hợp với tấn_công ngoại_giao , ra_sức xây_dựng lực_lượng quân_sự và chính_trị , phát_triển chiến_lược tấn_công một_cách toàn_diện , liên_tục và mạnh_mẽ , làm thất_bại các mục_tiêu và biện_pháp phòng_ngự của địch , đánh_bại âm_mưu kết_thúc chiến_tranh trên thế mạnh và chủ_trương phi mỹ hoá chiến_tranh của chúng , đánh cho mỹ phải rút hết quân , đánh cho 